In [1]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict as ddict

In [2]:
path = '/nfs/research/regan/data_propara/data/corenlp_propara_v9.json'

In [3]:
with open(path, 'r') as f:
    data = json.load(f)
len(data)

3521

In [4]:
def make_df_for_test(data, only_physical=True):
    
    all_items = []
    
    for k, v in data.items():
        
        if 'metadata' not in k:
                
            if v["fd1"] != '':

                sentence = v['example']

                features = v['features']
                for it in features:
                    if 'q_value' in it:
                        if it['q_value']!='':

                            entity = it['text']
                            q_value = it['q_value']

                            sent_with_entity = "<s> " + sentence + " </s> </s> " + entity + " </s>"

                            item = {"sentence":sent_with_entity,'+MER': 0, '-MER':0, 'COS':0, 'DES':0, 'EXIST':0, 
                                                        'INTL':0, 'MOT':0, 'VOL': 0}

                            if "::" in q_value:
                                q_values = q_value.split("::")
                                for q in q_values:
                                    if 'PROP' in q:
                                        item['COS'] = 1
                                    else:
                                        item[q] = 1
                            else:
                                if 'PROP' in q_value:
                                    item['COS'] = 1
                                else:
                                    item[q_value] = 1

                    #         if only_physical and d["type"] == "physical":
                    #             all_items.append(item)
                    #         else:
                            all_items.append(item)
        
    return all_items


    
all_items = make_df_for_test(data, only_physical=False)

df_test = pd.DataFrame(all_items)

In [5]:
df_test.head()

,sentence,+MER,-MER,COS,DES,EXIST,INTL,MOT,VOL
0,<s> Plants die </s> </s> Plants </s>,0,0,1,0,0,0,0,0
1,<s> They are buried in sediment </s> </s> They...,1,0,0,0,0,0,0,0
2,<s> They are buried in sediment </s> </s> sedi...,0,0,0,0,1,0,0,0
3,<s> Bacteria is buried in sediment </s> </s> B...,1,0,0,0,0,0,0,0
4,<s> Bacteria is buried in sediment </s> </s> s...,0,0,0,0,1,0,0,0


In [6]:
len(df_test)

2370

In [7]:
def check_if_each_instance_has_qvalue(df):
    
    count_of_qvalues = []
    
    for idx, row in df.iterrows():
        cnt = 0
        for i in range(1, 9):
            if row[i]==1:
                cnt+=1
        count_of_qvalues.append(cnt)
        
    
    print("number of instances:", len(count_of_qvalues)) 
    print("max count:", np.max(count_of_qvalues)) 
    print("min count:", np.min(count_of_qvalues)) 
    print("avg count:", np.mean(count_of_qvalues))            
            
check_if_each_instance_has_qvalue(df_test)

number of instances: 2370
max count: 2
min count: 1
avg count: 1.008860759493671


In [8]:
json_outfile_path = '../data/propara_q_value_v2.json'

# with open(json_outfile_path, 'w') as outfile:
#     json.dump(all_items, outfile)

In [10]:
count_volitional = df_test['VOL'].sum()
print(count_volitional)
print(count_volitional/len(df_test))

72
0.030379746835443037
